In [ ]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval

In [ ]:
# Import all the necessary classes and initialize the tokenizer and model.
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [ ]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='te'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

In [ ]:
small_train_dataset = raw_datasets["train"].select(range(20000))

# Print the length of the small dataset
print(len(small_train_dataset))
small_train_dataset.column_names

20000


['tokens', 'ner_tags']

In [ ]:
# let's print an instance of dataset
idx=10
rec=small_train_dataset[idx]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('{}\t{}'.format(w,t))


ఈ	0
సినిమా	0
కూడా	0
మంచి	0
హిట్	0
అయినది	0
.	0


In [ ]:
column_names = small_train_dataset.column_names
print(column_names)

features = small_train_dataset.features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [ ]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [ ]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [ ]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

model=model.to("cuda")

In [ ]:
pip install accelerate -U

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
train_dataset = small_train_dataset
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

In [ ]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

<ipython-input-16-435f98c90f62>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True
# Define training arguments
num_epochs=3
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=num_epochs,  # Set the number of epochs
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy='steps',
    eval_steps=100,
    logging_dir='./logs',
    logging_steps=10,
    disable_tqdm=False,
    load_best_model_at_end=True,  # Set load_best_model_at_end to True
    learning_rate=0.0001,  # Set the learning rate here
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)




/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=100,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_l

In [ ]:
import numpy as np

In [ ]:
train_result = trainer.train()
metrics = train_result.metrics

Step,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
100,0.177500,0.293546,0.744015,0.773946,0.758685,1044,0.636767,0.691418,0.662968,1037,0.753128,0.780938,0.766781,2004,0.720280,0.756426,0.737910,0.916994
200,0.213200,0.283611,0.791841,0.725096,0.757000,1044,0.631068,0.689489,0.658986,1037,0.767595,0.756487,0.762001,2004,0.735236,0.731457,0.733342,0.916093
300,0.164300,0.300783,0.755747,0.755747,0.755747,1044,0.651643,0.669238,0.660324,1037,0.728782,0.788423,0.757430,2004,0.716156,0.749816,0.732600,0.917391
400,0.169000,0.280933,0.729401,0.746169,0.737689,1044,0.586460,0.693346,0.635440,1037,0.765734,0.764970,0.765352,2004,0.705540,0.741983,0.723303,0.915697
500,0.252700,0.292777,0.691189,0.773946,0.730230,1044,0.692938,0.652845,0.672294,1037,0.756505,0.768962,0.762683,2004,0.723404,0.740759,0.731979,0.915552
600,0.333000,0.256301,0.735401,0.772031,0.753271,1044,0.688325,0.653809,0.670623,1037,0.741981,0.784930,0.762852,2004,0.727684,0.748348,0.737871,0.918219
700,0.262400,0.261924,0.746896,0.749042,0.747967,1044,0.640294,0.671167,0.655367,1037,0.770367,0.754990,0.762601,2004,0.729868,0.732191,0.731028,0.916814
800,0.166500,0.271189,0.742723,0.757663,0.750119,1044,0.673611,0.654773,0.664059,1037,0.756895,0.739521,0.748107,2004,0.732324,0.722644,0.727452,0.916309
900,0.295800,0.256300,0.745024,0.752874,0.748928,1044,0.686408,0.681774,0.684083,1037,0.739456,0.787425,0.762687,2004,0.727898,0.751775,0.739644,0.919193
1000,0.291300,0.258744,0.775362,0.717433,0.745274,1044,0.724138,0.627772,0.672521,1037,0.755898,0.783433,0.769419,2004,0.753425,0.727050,0.740002,0.918616


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [ ]:

metrics = trainer.evaluate()

trainer.log_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        1.2
  eval_LOC_f1             =     0.7453
  eval_LOC_number         =       1044
  eval_LOC_precision      =     0.7754
  eval_LOC_recall         =     0.7174
  eval_ORG_f1             =     0.6725
  eval_ORG_number         =       1037
  eval_ORG_precision      =     0.7241
  eval_ORG_recall         =     0.6278
  eval_PER_f1             =     0.7694
  eval_PER_number         =       2004
  eval_PER_precision      =     0.7559
  eval_PER_recall         =     0.7834
  eval_loss               =     0.2587
  eval_overall_accuracy   =     0.9186
  eval_overall_f1         =       0.74
  eval_overall_precision  =     0.7534
  eval_overall_recall     =     0.7271
  eval_runtime            = 0:00:23.97
  eval_samples_per_second =    112.628
  eval_steps_per_second   =       7.05


In [ ]:
test_dataset = raw_datasets['test']
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=6,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

In [ ]:

# Evaluate on the test dataset
metrics = trainer.evaluate(test_dataset)

trainer.log_metrics("test", metrics)

***** test metrics *****
  epoch                   =        1.2
  eval_LOC_f1             =     0.7886
  eval_LOC_number         =        483
  eval_LOC_precision      =     0.8372
  eval_LOC_recall         =     0.7453
  eval_ORG_f1             =      0.668
  eval_ORG_number         =        263
  eval_ORG_precision      =        0.7
  eval_ORG_recall         =     0.6388
  eval_PER_f1             =     0.8297
  eval_PER_number         =        609
  eval_PER_precision      =     0.8438
  eval_PER_recall         =     0.8161
  eval_loss               =      0.205
  eval_overall_accuracy   =     0.9364
  eval_overall_f1         =     0.7842
  eval_overall_precision  =     0.8141
  eval_overall_recall     =     0.7565
  eval_runtime            = 0:00:07.50
  eval_samples_per_second =    112.915
  eval_steps_per_second   =      7.066


In [ ]:
from datasets import Dataset, Features, ClassLabel, Value, Sequence

# Assuming your data is structured like this:
data = [
{   "tokens": [     "అయితే",     "ఈ",     "సారి",     "అనూహ్యంగా",     "కోటి",     "యాభై",     "లక్షల",     "పలకడంతో",     "ఆ",     "డబ్బును",     "ఏం",     "చేయాలో",     "ఇంకా",     "నిర్ణయించుకోలేదని",     "గోయట్",     "చెప్పుకొచ్చాడు"   ],   "ner_tags": [     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "B-PER",     "O",     "O"   ] },
{   "tokens": [     "ప్రజల",     "మనోభావాలు,",     "ఎమోషన్స్",     "కి",     "అంతగా",     "ప్రాముఖ్యత",     "ఇవ్వకుండా",     "తానుఅనుకున్న",     "పనులు",     "చక్కబెట్టేవారు",   " ."   ],   "ner_tags": [     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",  "O"   ] },
{   "tokens": [     "అప్పుడు,",     "మీరు",     "చివరకు",     "చూస్తున్న",     "పెరుగుదల",     "అలవాటు",     "రకం",     "ప్రోత్సహించడానికి",     "న్యాయమైన",     "కట్స్",     "తయారు",  "."  ],   "ner_tags": [     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O", "O"   ] },
{   "tokens": [ "ఎన్నికల", "ప్రకటన", "వెలువడినది", "లగాయతు", "సీట్ల", "పంపకం", "కోసం", "కొట్టుకోవడమే", "మహాకూటమికి", "సరిపోతున్నది", "ప్రతిరోజూ", "ఉదయం", "ఇంట్లో", "టిఫిన్", "చెయ్యడం", "గోల్కొండ", "రిసారట్స్కు", "వెళ్లి", "పిచ్చాపాటీ", "మాట్లాడుకుని", "భోజనం", "చెయ్యడం", "ఇంటికి", "వెళ్లడం", "మరునాడు", "మళ్ళీ", "అదే", "తంతు", "కాంగ్రెస్", "పార్టీకి", "కార్యకర్తలు", "తక్కువ", "నాయకులు", "ఎక్కువ", "సర్పంచ్", "గా", "కూడా", "చేయనివాడు", "ముఖ్యమంత్రి", "కావాలనుకుంటారు", "ఆ", "పార్టీలో", "అందుకే", "ఎప్పుడు", "ఎన్నికలు", "వచ్చినా", "కాంగ్రెస్", "పార్టీలో", "అంతర్గత", "యుద్ధం", "మొదలవుతుంది", "వారికి", "వారే", "వెన్నుపోట్లు", "పొడుచుకుంటారు", "తమకు", "ముప్ఫయి", "సీట్లు", "కావాలని", "టీజేఎస్", "తమకు", "నలభై", "కావాలని", "తెలుగుదేశం", "కాంగ్రెస్", "పార్టీకి", "హెచ్చరికలు", "జారీ", "చేస్తున్నాయి", "లేకపోతె", "తమ", "దారి", "తాము", "చూసుకుంటామని", "బెదిరింపులకు", "దిగుతున్నాయి", "ఆ", "రెండు", "పార్టీలకే", "అరవై", "డ్బ్బై", "స్థానాలు", "ఇచ్చి", "ఇక", "కాంగ్రెస్", "పోటీ", "చేసేది", "ఎన్ని", "స్థానాలలో", "నవ్వు", "రావడం", "లేదూ", "ఎన్నికలు", "అయ్యేంతవరకూ", "పరస్పరం", "కలహించుకోవడంలోనే", "కాలక్షేపం", "మహాకూటమి", "ఎన్నికల్లో", "పోటీ", "చేసి", "విజయం", "సాధించడం", "అయ్యేపనేనా" ], "ner_tags": [ "O", "O", "O", "O", "B-LOC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "I-LOC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-ORG", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-ORG", "O", "O", "O", "B-ORG", "B-ORG", "O", "O", "O", "O", "O", "O", "O", "O", "B-ORG", "O", "O", "O", "O", "B-ORG", "I-ORG", "O", "O", "O", "O", "O", "O", "B-ORG", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
{ "tokens": [ "మెగా", "పవర్", "స్టార్", "రామ్", "చరణ్", ",", "ఊరమాస్", "డైరెక్టర్", "బోయపాటి", "శ్రీను", "కాంబినేషన్‌లో", ",", "ఫ్యామిలీ", ",", "లవ్", "అండ్", "యాక్షన్", "ఎంటర్‌టైనర్‌గా", "రూపొందిన", "వినయ", "విధేయ", "రామ్", ",", "జనవరి", "11న", "వరల్డ్", "వైడ్", "గ్రాండ్", "రిలీజ్‌కి", "రెఢీ", "అయిపోయింది", ".", "డి.వి.వి.", "దానయ్య", "నిర్మించగా", ",", "కైరా", "అద్వాణీ", "హీరోయిన్‌గా", "నటించింది" ], "ner_tags": [ "O", "O", "O", "B-PER", "I-PER", "O", "O", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "I-PER", "O", "O" ] },
{ "tokens": [ "అది,", "పొందుటకు", ",",  "మేము", "ప్రధాన", "ద్వారం", "నుండి", "కుడి", "చెయ్యి", "మరియు", "నిరంతరం", "పెరుగుతుంది", "." ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
  { "tokens": [ "లోతైన", "సమావేశంలో", ",", "సారాంశం", "మానవ", "శరీరంలో", "నుండి", "వేరు", ",", "అస్తిత్వ", "మరణం", "అని", "పిలువబడుతుంది", "." ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
 { "tokens": [ "చాటుసాహిత్యంలో", "తరచూ", "కనిపించే", "లక్షణం", "–", "పద్యాల", "గురించీ", ",", "కవుల", "గురించీ", "జనవ్యవహారంలో", "వ్యాప్తమైయ్యే", "కథలు", "." ], "ner_tags": [ "O", "O", "O",  "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
{ "tokens": [ "బ్రేకింగ్", ":", "మళ్ళీ", "వైసీపీలోకి", "ఎంట్రీ", "ఇస్తున్న", "కీలక", "నేత", "..", "షాక్‌లో", "టీడీపీ", "." ], "ner_tags": [ "O", "O", "O", "B-ORG", "O", "O", "O", "O", "O", "O", "B-ORG", "O"] },
 { "tokens": [ "ప్రామాణిక", "బ్యాంకు", "రుణ", "జారీ", "కాకుండా", ",", "క్రెడిట్", "యొక్క", "లైన్", "ఆర్థిక", "సంస్థల", "రుణగ్రహీతలు", "మరింత", "ఆకర్షణీయంగా", ",", "అలాగే", "ఉంది", "." ], "ner_tags": [ "O", "B-ORG", "O", "O", "O", "O", "O", "O", "O", "B-ORG", "I-ORG", "O", "O", "O", "O", "O", "O", "O" ] },
 { "tokens": [ "వారి", "ప్రకాశవంతమైన", "నక్షత్రాలు", "సిరియస్", "మరియు", "Procyon", "ఏ", "ఇతర", "తో", "తికమక", "కష్టం", "." ], "ner_tags": [ "O", "O", "O", "B-PER", "O", "B-PER", "O", "O", "O", "O", "O", "O" ] },
 { "tokens": [ "పీటర్స్బర్గ్", "పతనం", "తరువాత", "లీ", "యొక్క", "వెనుకవైపు", "ఉన్న", "సైనికదళాన్ని", "కొనసాగించడంతో", ",",  "రైట్", "మరియు", "VI", "కార్ప్స్", "మళ్లీ", "షెరిడాన్", "దిశగా", "వచ్చారు", "." ], "ner_tags": [ "B-PER", "O", "O", "B-PER", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "B-PER",  "O", "B-LOC", "O", "O", "O" ] },
 { "tokens": [ "దానిపై", "హైడ్", "ప్రదర్శన", "లక్షణాలు", "ఒక", "ప్రాథమిక", "వివరణ", "తో", "ప్రారంభం", "కావాలి", "." ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
{ "tokens": [ "మోరెనా", "జిల్లా", "జౌరా", "ప్రాంతంలో", "జరిగిన", "ఒక", "ఎన్నికల", "సభలో", "రాహుల్", "మాట్లాడుతూ", "ఎంజే", "అక్బర్‌పై", "పలువురు", "మహిళా", "జర్నలిస్టులు", "చేసిన", "లైంగిక", "దాడి", "ఆరోపణలను", "దృష్టిలో", "పెట్టుకొని", "మోదీ", "ప్రభుత్వాన్ని", "నిలదీశారు", "." ], "ner_tags": [ "B-LOC", "O", "B-LOC", "O", "O", "O", "O", "O", "B-PER", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "O" ] },

 #  { "tokems": [ "కాన్స్టాంటినోపుల్లో", "బోహెంండ్", "ముఖ్యంగా", "స్వాగతం", "లేదు", "ఎందుకంటే", "అతని", "తండ్రి", ",", "రాబర్ట్", "గైకార్డ్", ",", "బైజాంటైన్", "సామ్రాజ్యాన్ని", "ఆక్రమించి", ",", "డియర్హచ్యూమ్", "మరియు", "కోర్ఫు", "నగరాలను", "స్వాధీనం", "చేసుకున్నాడు", "." ], "ner_tags": [ 6, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 6, 0, 0, 0, 6, 0, 6, 0, 0, 0, 0 ] },
 #   { "tokems": [ "శిఖరం", "(", "ఉత్తమ", "సమయం", "-", "సాయంత్రం", "ముఖ్యంగా", "“", "వర్షం", "కింద", "“", ")", "యొక్క", "వేడి", "మానుకోండి", ";" ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
 #  { "tokems": [ "ఏదైనా", "ఒక", "నీటి", "పారుదల", "ప్రాజెక్టుకు", "జాతీయ", "హోదాను", "ఇవ్వాలని", "కోరే", "అవకాశం", "ఉంది", "." ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
 #{ "tokems": [ "ప్రైవేట్స్‌", "పోల్స్‌ను", "బట్టి", "చూస్తే", "వితిక", "పరిస్థితి", "మాత్రం", "దారుణంగా", "ఉన్నట్లు", "తెలుస్తోంది", "." ], "ner_tags": [ 0,0,0,0,0,0,0,0,0,0,0 ] },
 # { "tokems": [ "బంగ్లా", "ఓపెనర్", "షాద్‌మన్‌", "ఇస్లామ్‌.", "(", "29", ")", "ఫర్వాలేదనిపించగా", "...", "మరో", "ఓపెనర్", "ఇమ్రుల్‌", "(", "4", ")", "నిరాశపరిచాడు", "." ], "ner_tags": [ "B-LOC", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "O", "O" ] },
 # { "tokems": [ "అయితే", "వాటికి", "హైనెక్‌", "బ్లవుజులు", ",", "కాలర్స్‌", ",", "ఫుల్‌స్లీవ్స్‌", ",", "క్లోజ్డ్‌", ",", "బోట్‌", ",", "బ్యాక్‌హైనెక్", "‌,", "ఫ్రంట్‌", "రౌండ్‌నెక్‌", "బ్లవుజ్‌లు", "బాగుంటాయి", "." ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
 # {    "tokems": [     "4",     ")",     "పైన",     "చెప్పిన",     "వి",     "అనగా",     "కలిపి",     "పెట్టుకున్న",     "రవ్వ",     ",",     "ఉడికించిన",     "ఆలూ",     ",",     "కట్",     "చెసి",     "పెట్టకున్నవి",     "అన్ని",     "ఇంకా",     "జిలకర్ర",     ",",     "ఉప్పు",     "రుచి",     "కి",     "సరిపడ",     ".",     "వెసి",     "ముద్ద",     "లా",     "కలుపుకోవాలీ",     "."   ],   "ner_tags": [     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O"   ] },
 #{   "tokems": [     "సినిమాలో",     "కంటెంట్",     "అంతంతమాత్రమే",     "అయినప్పటికీ",     "..",     "ఈ",     "మాత్రం",     "వసూళ్లు",     "వచ్చాయంటే",     "అది",     "ఎన్టీఆర్",     "పెర్ఫామెన్స్‌",     "వల్లే",     "."   ],   "ner_tags": [     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "B-PER",     "O",     "O"   ] },
  #{   "tokems": [     "‘",     "స్టార్స్",     "‌’",     "(",     "స్కీమ్‌",     "ఫర్‌",     "ట్రాన్స్‌ఫర్మేషన్‌",     "అండ్‌",     "అడ్వాన్స్‌డ్‌",     "రీసెర్చ్‌",     "ఇన్‌",     "బేసిక్‌",     "సైన్సెస్",     ")",     ",",     "ఐఐఎస్‌సీ",     "బెంగళూరు",     "ద్వారా",     "అధ్యయన",     "కార్యక్రమాలను",     "మంజూరు",     "చేస్తున్నారు",     "."   ],   "ner_tags": [     "O",     "O",     "O",     "O",     "B-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "O",     "O",     "B-ORG",     "B-LOC",     "O",     "O",     "O",     "O",     "O",     "O",     "O"   ] }
    # More records...
]

# Transform your list of dictionaries into the expected format
transformed_data = {
    "tokens": [item["tokens"] for item in data],
    "ner_tags": [item["ner_tags"] for item in data],
}

# Define the features of your dataset, aligning with your manual data structure
features = Features({
    'tokens': Sequence(feature=Value(dtype='string')),
    'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))
})

# Create a Dataset object from your transformed data
question1_data = Dataset.from_dict(transformed_data, features=features)

# Now you can access column_names and features just like your training dataset
column_names = question1_data.column_names
print(column_names)

dataset_features = question1_data.features
print(dataset_features)


['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [ ]:
test = question1_data
test = test.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=6,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset (num_proc=6):   0%|          | 0/14 [00:00<?, ? examples/s]

In [ ]:

# Evaluate on the test dataset
metrics = trainer.evaluate(test)

trainer.log_metrics("test", metrics)

***** test metrics *****
  epoch                   =        1.2
  eval_LOC_f1             =     0.3333
  eval_LOC_number         =          5
  eval_LOC_precision      =        1.0
  eval_LOC_recall         =        0.2
  eval_ORG_f1             =     0.4706
  eval_ORG_number         =         11
  eval_ORG_precision      =     0.6667
  eval_ORG_recall         =     0.3636
  eval_PER_f1             =     0.5385
  eval_PER_number         =         14
  eval_PER_precision      =     0.5833
  eval_PER_recall         =        0.5
  eval_loss               =      0.335
  eval_overall_accuracy   =     0.9034
  eval_overall_f1         =     0.4898
  eval_overall_precision  =     0.6316
  eval_overall_recall     =        0.4
  eval_runtime            = 0:00:00.16
  eval_samples_per_second =     83.646
  eval_steps_per_second   =      5.975
